In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

# Plant location


Le problème de localisation d'usine (*plant location*) se caractérise comme suit: des usines $i$ peuvent être construites à un coût $f_i$ à des emplacements donnés (chaque usine a un site qui lui est propre). On suppose que des clients $j$ doivent être livrés par ces usines, et que chaque usine a la capacité de livrer tous les clients. Le coût de livraison depuis l'usine $i$ vers le client $j$ est noté $c_{ij}$.

Le problème est de déterminer quelles usines doivent être construites et comment se fait la livraison des clients (un client peut être livré depuis plusieurs usines), afin de minimiser le coût global.


Pour modéliser le problème, nous définissons les variables suivantes:
> - $y_{i}=1$ si l'usine $P_i$ est construite, 0 sinon
> - $x_{ij}$ représente le pourcentage de la commande du client $j$ livrée depuis l'usine $i$ ($x_{ij}\in [0,1]$)
    
On doit vérifier que toute la commande est bien livrée au client depuis les différentes usines.

## 1. Stratégie de décomposition
Écrire le problème dans un cas à deux usines et deux clients. Écrire la matrice de contraintes et en déduire le type de décomposition qui serait pertinent.

## 2. Modèle complet
Modéliser et résoudre en programmation linéaire en nombres entier le problème à 3 usines et 3 clients donné ci-dessous.

In [2]:
from docplex.mp.model import Model
import numpy as np

# Données
NbUsines = 3
NbClients = 3

CoutOuverture = [20, 20, 20]
CoutLivraison = np.array([[15, 1, 2],
 [1, 16, 3],
 [4, 1, 17]])

Usines = range(NbUsines)
Clients = range(NbClients)

In [3]:
# Modèle
pl = Model(name='Plant location')

# Variables: décision de construction d'une usine
y = pl.binary_var_list(Usines, name='Y')

# Variables: livraison d'une usine à un client
x = pl.continuous_var_matrix(Usines, Clients, name='X')

# Contraintes: livraison du client j assurée depuis les différentes usines
for j in Clients:
    pl.add_constraint(pl.sum(x[i, j] for i in Usines) == 1)

# Contraintes: livraison depuis une usine seulement si elle est ouverte
for i in Usines:
    for j in Clients:
        pl.add_constraint(x[i, j] <= y[i])

# Objectif
CoutTotalOuverture = pl.sum(y[i]*CoutOuverture[i] for i in Usines)
CoutTotalLivraison = pl.sum(x[i,j]*CoutLivraison[i,j] for i in Usines for j in Clients)
pl.minimize(CoutTotalOuverture + CoutTotalLivraison)

print(pl.export_as_lp_string())

pl.solve()

pl.print_solution()

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Plant location

Minimize
 obj: 20 Y_0 + 20 Y_1 + 20 Y_2 + 15 X_0_0 + X_0_1 + 2 X_0_2 + X_1_0 + 16 X_1_1
      + 3 X_1_2 + 4 X_2_0 + X_2_1 + 17 X_2_2
Subject To
 c1: X_0_0 + X_1_0 + X_2_0 = 1
 c2: X_0_1 + X_1_1 + X_2_1 = 1
 c3: X_0_2 + X_1_2 + X_2_2 = 1
 c4: X_0_0 - Y_0 <= 0
 c5: X_0_1 - Y_0 <= 0
 c6: X_0_2 - Y_0 <= 0
 c7: X_1_0 - Y_1 <= 0
 c8: X_1_1 - Y_1 <= 0
 c9: X_1_2 - Y_1 <= 0
 c10: X_2_0 - Y_2 <= 0
 c11: X_2_1 - Y_2 <= 0
 c12: X_2_2 - Y_2 <= 0

Bounds
0 <= Y_0 <= 1
0 <= Y_1 <= 1
0 <= Y_2 <= 1

Binaries
 Y_0 Y_1 Y_2
End

objective: 38.000
  Y_0=1
  X_0_0=1.000
  X_0_1=1.000
  X_0_2=1.000


## 3. Décomposition de Benders (Implémentation avec un seul sous-problème)

In [4]:
# Définition du problème maître restreint
pl_master= Model(name='Problème maître restreint')
y = pl_master.binary_var_list(Usines, name='Y')
z = pl_master.continuous_var(name='Z')
pl_master.minimize(pl_master.sum(y[i]*CoutOuverture[i] for i in Usines) + z)
print("Problème maître restreint\n")
print("-------------------------")
print(pl_master.export_as_lp_string())

# Définition des contraintes du sous-problème dual
pl_worker = Model(name='Sous-problème de Benders dual')
pl_worker.parameters.preprocessing.presolve.set(0)

u = pl_worker.continuous_var_list(NbClients, lb=-pl_worker.infinity, ub=pl_worker.infinity, name='U')
v = pl_worker.continuous_var_matrix(NbUsines, NbClients, lb=-pl_worker.infinity, ub=0, name='V')
for j in Clients:
    for i in Usines:
        pl_worker.add_constraint(u[j]+v[i, j] <= CoutLivraison[i][j])

# Boucle de l'algo
# -----------------
n=0
worker_obj = pl_worker.infinity
epsilon = 1E-6
boucle = True 
optimal = False

while boucle:
    n = n+1
    print("Itération ",n)
    print("-------------")
    
    # Résolution du problème maître restreint
    master_sol = pl_master.solve()
    if master_sol is None :
        print("Problème non réalisable")
        break
    master_obj = master_sol.get_objective_value()
    print("master:",master_obj)
    sy = [master_sol.get_value(y[i]) for i in Usines]
    sz  = master_sol.get_value(z)
    print("y=",sy) 
    print("z=",sz)
    
    #Coefficients des coupes de Benders associés aux elements du vecteur y et coefficient constant --- utiles pour afficher les coupes de facon lisible
    cut_y_coeffs = [0 for i in Usines]
    cut_constant = 0
    
    # Résolution du sous-pb dual
    pl_worker.maximize(pl_worker.sum(u[j] for j in Clients) + pl_worker.sum(sy[i]*v[i, j] for i in Usines for j in Clients))
    print("Sous-problème dual -- iteration",n)
    print("------------------")
    print(pl_worker.export_as_lp_string())
    worker_sol = pl_worker.solve()
    if pl_worker.solve_details.status == 'infeasible' :
        print("Problème dual non realisable --> STOP")
        boucle = False
    elif pl_worker.solve_details.status == 'unbounded' :            
        # Ajout d'une coupe de faisabilité
        print("Problème dual non borné")
        worker_obj = pl_worker.infinity
        ray = pl_worker.get_engine().get_cplex().solution.advanced.get_ray()
        for i in Usines:
            for j in Clients:
                cut_y_coeffs[i] += ray[NbClients+NbUsines*i+j]
        for j in Clients:
            cut_constant += ray[j]  
        pl_master.add_constraint(pl_master.sum(ray[j] for j in Clients)+
                                 pl_master.sum(y[i]*ray[NbClients+NbUsines*i+j] for i in Usines for j in Clients)
                                 <=0)
        print("Coupe de faisabilité:", end=" ")
        for i in Usines:
            print(cut_y_coeffs[i],'*Y_',i, end=" ", sep='')
        print('+',cut_constant," <= 0")
    else :            
        # Ajout d'une coupe d'optimalité au problème maître restreint
        worker_obj = worker_sol.get_objective_value()
        if worker_obj - sz > epsilon:
            su = [worker_sol.get_value(u[j]) for j in Clients]
            sv = [worker_sol.get_value(v[i, j]) for i in Usines for j in Clients]
            for i in Usines:
                for j in Clients:
                    cut_y_coeffs[i] += sv[NbUsines*i+j]
            for j in Clients:
                cut_constant += su[j]  
            pl_master.add_constraint(pl_master.sum(su[j] for j in Clients)+
                                     pl_master.sum(y[i]*sv[NbUsines*i+j] for i in Usines for j in Clients)
                                     <=z)
            print("Coupe d'optimalité:", end=" ")
            for i in Usines:
                print(cut_y_coeffs[i],'*Y_',i, end=" ", sep='')
            print('+',cut_constant," <= z")
                
    # Sortie
    if boucle:
        print("w =",worker_obj)
        print("sz =", sz)
        print("Borne inf:",master_obj)
        print("Borne sup:",worker_obj+pl_master.sum(sy[i]*CoutOuverture[i] for i in Usines))
        boucle = False
        optimal = True
        for j in Clients:
            if worker_obj - sz > epsilon:
                boucle = True
                optimal = False
                break
                
if optimal:
    print("master:",master_obj)
print("Fini !")


Problème maître restreint

-------------------------
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Probl_xe8me ma_xeetre restreint

Minimize
 obj: 20 Y_0 + 20 Y_1 + 20 Y_2 + Z
Subject To

Bounds
0 <= Y_0 <= 1
0 <= Y_1 <= 1
0 <= Y_2 <= 1

Binaries
 Y_0 Y_1 Y_2
End

Itération  1
-------------
master: 0.0
y= [0, 0, 0]
z= 0
Sous-problème dual -- iteration 1
------------------
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Sous-probl_xe8me de Benders dual

Maximize
 obj: U_0 + U_1 + U_2
Subject To
 c1: U_0 + V_0_0 <= 15
 c2: U_0 + V_1_0 <= 1
 c3: U_0 + V_2_0 <= 4
 c4: U_1 + V_0_1 <= 1
 c5: U_1 + V_1_1 <= 16
 c6: U_1 + V_2_1 <= 1
 c7: U_2 + V_0_2 <= 2
 c8: U_2 + V_1_2 <= 3
 c9: U_2 + V_2_2 <= 17

Bounds
      U_0 Free
      U_1 Free
      U_2 Free
-inf <= V_0_0 <= 0
-inf <= V_0_1 <= 0
-inf <= V_0_2 <= 0
-inf <= V_1_0 <= 0
-inf <= V_1_1 <= 0
-inf <= V_1_2 <= 0
-inf <= V_2_0 <= 0
-inf <= V_2_1 <= 0
-inf <= V_2_2 <= 0
End

Problèm

## 4. Décomposition de Benders (Implémentation avec un sous-problème par client)

In [8]:
# Définition du problème maître restreint
pl_master= Model(name='Problème maître restreint')
y = pl_master.binary_var_list(Usines, name='Y')
# Une variable z par sous-probleme 
z = pl_master.continuous_var_list(Clients,name='Z')
pl_master.minimize(pl_master.sum(y[i]*CoutOuverture[i] for i in Usines) + pl_master.sum(z[j] for j in Clients))
print()
print("Problème maître restreint")
print("-------------------------")
print(pl_master.export_as_lp_string())

# Définition des contraintes des sous-problèmes duaux (un sous-problème par client)
pl_worker = [Model(name='Benders sous-problème dual client{}'.format(j)) for j in Clients]
for j in Clients:
    pl_worker[j].parameters.preprocessing.presolve.set(0)

u = [pl_worker[j].continuous_var(name='U', lb=-pl_worker[j].infinity, ub=pl_worker[j].infinity) for j in Clients]
v = [pl_worker[j].continuous_var_list(NbUsines, lb=-pl_worker[j].infinity, ub=0, name='V') for j in Clients]

for j in Clients:
    for i in Usines:
        pl_worker[j].add_constraint(u[j]+v[j][i] <= CoutLivraison[i][j])


# Boucle de l'algo
# -----------------
n=0
sz = [0]*NbClients
worker_obj = [pl_worker[j].infinity for j in Clients]
su = [0 for j in Clients for i in range(1+NbUsines)]
epsilon    = 1E-6
boucle = True 
optimal = False

while boucle:
    n = n+1
    print("Itération ",n)
    print("-------------")
    
    # Résolution du problème maître restreint
    master_sol = pl_master.solve()
    if master_sol is None :
        print("Problème non réalisable")
        break
    master_obj = master_sol.get_objective_value()
    print("master:",master_obj)
    sy = [master_sol.get_value(y[i]) for i in Usines]
    sz  = [master_sol.get_value(z[j]) for j in Clients]
    print("y=",sy) 
    print("z=",sz)
    
    # Résolution des sous-problèmes duaux
    for j in Clients: 
        pl_worker[j].maximize(u[j] + pl_worker[j].sum(sy[i]*v[j][i] for i in Usines))
        print("Sous-problème dual -- client",j," -- iteration",n)
        print("------------------")
        print(pl_worker[j].export_as_lp_string())
        #Coefficients des coupes de Benders associés aux elements du vecteur y et coefficient constant --- utiles pour afficher les coupes de facon lisible
        cut_y_coeffs = [0 for i in Usines]
        cut_constant = 0
        
        worker_sol = pl_worker[j].solve()
        if pl_worker[j].solve_details.status == 'infeasible' :
            print("Problème dual non réalisable --> STOP")
            boucle = False
            break
        elif pl_worker[j].solve_details.status == 'unbounded' :            
            # Ajout d'une coupe de faisabilité
            print("Problème dual non borné")
            worker_obj[j] = pl_worker[j].infinity
            ray = pl_worker[j].get_engine().get_cplex().solution.advanced.get_ray()
            for i in Usines:
                cut_y_coeffs[i] = ray[1+i]
            cut_constant = ray[0] 
            pl_master.add_constraint(ray[0]+pl_master.sum(y[i]*ray[1+i] for i in Usines)<=0)
            print("Coupe de faisabilité:", end=" ")
            for i in Usines:
                print(cut_y_coeffs[i],'*Y_',i, end=" ", sep='')
            print('+',cut_constant," <= 0")
            break            
        else :            
            # Ajout d'une coupe d'optimalité au problème maître restreint
            worker_obj[j] = worker_sol.get_objective_value()
            if worker_obj[j] - sz[j] > epsilon:
                su = worker_sol.get_value(u[j])
                sv = [worker_sol.get_value(v[j][i]) for i in range(NbUsines)]
                for i in Usines:
                    cut_y_coeffs[i] = sv[i]
                cut_constant = su 
                pl_master.add_constraint(su+pl_master.sum(y[i]*sv[i] for i in Usines)<=z[j])
                print("Coupe d'optimalité:", end=" ")
                for i in Usines:
                    print(cut_y_coeffs[i],'*Y_',i, end=" ", sep='')
                print('+',cut_constant," <= z[",j,"]")
                break
                
    # Sortie
    if boucle:
        print("w =",worker_obj)
        print("sz =", sz)
        print("Borne inf:",master_obj)
        print("Borne sup:",pl_worker[j].sum(worker_obj[j] for j in Clients)+pl_master.sum(sy[i]*CoutOuverture[i] for i in Usines))
        boucle = False
        optimal = True
        for j in Clients:
            if worker_obj[j] - sz[j] > epsilon:
                boucle = True
                optimal = False
                break
    print()

if optimal:
    print("master:",master_obj)
print("Fini !")



Problème maître restreint
-------------------------
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Probl_xe8me ma_xeetre restreint

Minimize
 obj: 20 Y_0 + 20 Y_1 + 20 Y_2 + Z_0 + Z_1 + Z_2
Subject To

Bounds
0 <= Y_0 <= 1
0 <= Y_1 <= 1
0 <= Y_2 <= 1

Binaries
 Y_0 Y_1 Y_2
End

Itération  1
-------------
master: 0.0
y= [0, 0, 0]
z= [0, 0, 0]
Sous-problème dual -- client 0  -- iteration 1
------------------
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Benders sous-probl_xe8me dual client0

Maximize
 obj: U
Subject To
 c1: U + V_0 <= 15
 c2: U + V_1 <= 1
 c3: U + V_2 <= 4

Bounds
      U Free
-inf <= V_0 <= 0
-inf <= V_1 <= 0
-inf <= V_2 <= 0
End

Problème dual non borné
Coupe de faisabilité: -1.0*Y_0 -1.0*Y_1 -1.0*Y_2 + 1.0  <= 0
w = [1e+20, 1e+20, 1e+20]
sz = [0, 0, 0]
Borne inf: 0.0
Borne sup: 300000000000000000000

Itération  2
-------------
master: 20.0
y= [1.0, 0, 0]
z= [0, 0, 0]
Sous-problème dual -- client 0  -- i

### 5. Décomposition de Benders automatique de CPLEX


In [7]:
plb = Model(name='Plant location Benders CPLEX', log_output=True)

# Variables: décision de construction d'une usine
y = plb.binary_var_list(Usines, name='Y')

# Variables: livraison d'une usine à un client
x = plb.continuous_var_matrix(Usines, Clients, name='X')

# Contraintes: livraison du client j assurée depuis les différentes usines
for j in Clients:
    plb.add_constraint(plb.sum(x[i, j] for i in Usines) == 1)

# Contraintes: livraison depuis une usine seulement si elle est ouverte
for i in Usines:
    for j in Clients:
        plb.add_constraint(x[i, j] <= y[i])

# Objectif
CoutTotalOuverture = plb.sum(y[i]*CoutOuverture[i] for i in Usines)
CoutTotalLivraison = plb.sum(x[i,j]*CoutLivraison[i,j] for i in Usines for j in Clients)
plb.minimize(CoutTotalOuverture + CoutTotalLivraison)

# Benders "automatique"
plb.parameters.preprocessing.presolve.set(0)
plb.parameters.preprocessing.reduce.set(0)
#plb.parameters.benders.strategy = 3

# Benders guidé: choix problème maître(0)/sous-problème (1)
plb.parameters.benders.strategy = 1
for i in Usines:
    y[i].benders_annotation = 0
    for j in Clients:
        x[i,j].benders_annotation = 1

plb.print_information()

plb.solve()
plb.print_solution()


Model: Plant location Benders CPLEX
 - number of variables: 12
   - binary=3, integer=0, continuous=9
 - number of constraints: 12
   - linear=12
 - annotations: 12
   - variables: 12
 - parameters:
     parameters.benders.strategy = 1
     parameters.preprocessing.presolve = 0
     parameters.preprocessing.reduce = 0
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Reduce                    0
CPXPARAM_RandomSeed                              201703173
CPXPARAM_Benders_Strategy                        1
Found incumbent of value 78.000000 after 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0        0.0000                        Benders: 1        0         
      0     0       20.0000                        Benders: 1        1         
      0     0       32.8571                        B